## Imports


In order for us to read the files, we need to import certain modules. Pandas and altair allow us to plot the data in the picologger file

In [35]:
import os
import sys
import pandas as pd
import altair as alt
from datetime import datetime,date
import numpy as np

In [2]:
# List installed package versions
packList = ["pandas", "altair", "numpy", "itkdb"]
for p in packList:
    try:
        mod = __import__(p)
        print(mod.__name__ + " : " + mod.__version__)
    except ImportError:
        print(p + " not installed")

# Get the current working directory
thisDir = os.getcwd()

# List files in the parent directory
parentDir = os.path.abspath(os.path.join(thisDir, os.pardir))
files_in_parent = os.listdir(parentDir)
print("Files in parent directory:", files_in_parent)

pandas : 1.3.4
altair : 4.2.0
numpy : 1.22.3
itkdb : 0.4.8
Files in parent directory: ['.DS_Store', 'picolog_folder', 'requirements.txt', 'output_file.csv', 'README.md', '.gitignore', 'HV_LV_TEST.json', '.git', 'notebooks']


## Read in Data

In [3]:
### list files in above directory
thisDir = os.getcwd()
os.listdir(thisDir+"/../")

['.DS_Store',
 'picolog_folder',
 'requirements.txt',
 'output_file.csv',
 'README.md',
 '.gitignore',
 'HV_LV_TEST.json',
 '.git',
 'notebooks']

## Format raw data
The following data takes in all of the files from the picologger folder,performs necessary calculations on them and puts them all in one csv file.

In [4]:
# Timestamp converter function
def TimeStampConverter(inStr, inPat):
    timeObj = None
    if inStr == "now":
        timeObj = datetime.now()
    else:
        timeObj = datetime.strptime(inStr, inPat)
    return timeObj.strftime("%Y-%m-%dT%H:%M:%SZ")

In [59]:
### derive values from picoLogger raw data
# Perform calculations function
def perform_calculations(input_folder, output_file, timestamp_str):
    all_data = []

    timestamp = TimeStampConverter(timestamp_str, "%Y-%m-%d %H:%M:%S")

    ### loop over files
    for filename in sorted(os.listdir(input_folder)):

        # skip if not csv
        if not filename.endswith('.csv'):
            continue

        file_path = os.path.join(input_folder, filename)
        print("working on:",file_path)

        ### read in data
        data = pd.read_csv(file_path)

        ### prepare derive values
        derived ={}

        # Calculate Vin Drop and GND Drop
        derived['VIN_DROP'] = np.average(data['Vin+ Last (V)'] - data['Vin- Last (V)'])
        derived['GND_DROP'] =  np.average(data['GND+ Last (V)'] - data['GND- Last (V)'])
        
        # Calculate Resistance Vin(Ohms) and Total Resistance(mOhms)
        derived['R_VIN'] = derived['VIN_DROP'] / 10 / 5
        derived['R_GND'] = derived['GND_DROP'] / 10 / 5
        derived['EFFECTIVE_RESISTANCE'] = (derived['R_VIN'] + derived['R_GND']) * 1000
        
        # Calculate Capacitor Equivalent leakage current(nA)
        derived['HV_LEAKAGE']=  np.average(data['Capacitor leakage test Last (V)'])
        derived['LEAKAGE_CURRENT'] = ((derived['HV_LEAKAGE'] / 10) / (1 * 10 ** 6)) / 1000000
        
        # Calculate NTC value(Kohms)
        derived['NTC_VOLTAGE']=  np.average(data['NTC Last (V)'])
        derived['NTC_VALUE'] = 0.2 * 51 / derived['NTC_VOLTAGE']
                    
        # Prepare the dictionary with desired columns and values
        file_data = {
            'component': filename.replace('.csv',''),
            'componentType': 'PCB',
            'stage': 'PCB_QC',
            'testType': 'HV_LV_TEST',
            'institution': 'GL',
            'runNumber': '',
            'date': timestamp,
            'passed': 'true',
            'problems': 'false',
            'property1_value': 'B.masic',
            'property1_key': 'OPERATOR',
            'property2_value': 'INSTRUMENT',
            'property2_key': '',
            'property3_value': 'ANALYSIS_VERSION',
            'property3_key': 'picoScript: https://github.com/kwraight/picoLogger',
            'result1_key': 'VIN_DROP',
            'result1_value': derived['VIN_DROP'],
            'result2_key': 'GND_DROP',
            'result2_value': derived['GND_DROP'],
            'result3_key': 'EFFECTIVE_RESISTANCE',
            'result3_value': derived['EFFECTIVE_RESISTANCE'],
            'result4_key': 'HV_LEAKAGE',
            'result4_value': derived['HV_LEAKAGE'],
            'result5_key': 'LEAKAGE_CURRENT',
            'result5_value': derived['LEAKAGE_CURRENT'],
            'result6_key': 'NTC_VOLTAGE',
            'result6_value': derived['NTC_VOLTAGE'],
            'result7_key': 'NTC_VALUE',
            'result7_value': derived['NTC_VALUE']
        }
        ### append data
        all_data.append(file_data)
    
    ### convert all data to dataframe
    df_all=pd.DataFrame(all_data)
    # view for sanity
    display(df_all)

    ### save data if not empty
    if not df_all.empty:
        df_all.to_csv(output_file, index=False)
        print(f"Calculation completed. Output saved to {output_file}.")
    else:
        print(f"No data found :(")
        return None


In [60]:
### run conversion
input_folder = thisDir+"/../picolog_folder/"
output_file = thisDir+"/../output_file.csv"
timestamp_str = "2023-08-08 14:30:00"  # Replace this with your desired timestamp

perform_calculations(input_folder, output_file, timestamp_str)



working on: /Users/kwraight/repositories/picoLogger/notebooks/../picolog_folder/ASN_1.csv
working on: /Users/kwraight/repositories/picoLogger/notebooks/../picolog_folder/ASN_2.csv
working on: /Users/kwraight/repositories/picoLogger/notebooks/../picolog_folder/ASN_3.csv


,component,componentType,stage,testType,institution,runNumber,date,passed,problems,property1_value,...,result3_key,result3_value,result4_key,result4_value,result5_key,result5_value,result6_key,result6_value,result7_key,result7_value
0,ASN_1.csv,PCB,PCB_QC,HV_LV_TEST,GL,,2023-08-08T14:30:00Z,true,false,B.masic,...,EFFECTIVE_RESISTANCE,11.546667,HV_LEAKAGE,0.097333,LEAKAGE_CURRENT,9.733333e-15,NTC_VOLTAGE,1.300000,NTC_VALUE,7.846154
1,ASN_2.csv,PCB,PCB_QC,HV_LV_TEST,GL,,2023-08-08T14:30:00Z,true,false,B.masic,...,EFFECTIVE_RESISTANCE,11.580000,HV_LEAKAGE,0.106333,LEAKAGE_CURRENT,1.063333e-14,NTC_VOLTAGE,1.333667,NTC_VALUE,7.648088
2,ASN_3.csv,PCB,PCB_QC,HV_LV_TEST,GL,,2023-08-08T14:30:00Z,true,false,B.masic,...,EFFECTIVE_RESISTANCE,11.620000,HV_LEAKAGE,0.104667,LEAKAGE_CURRENT,1.046667e-14,NTC_VOLTAGE,1.357667,NTC_VALUE,7.512890


Calculation completed. Output saved to /Users/kwraight/repositories/picoLogger/notebooks/../output_file.csv.
